# Transportation model under varying material sizes

# 

In [194]:
from pulp import LpProblem, LpVariable, LpInteger, lpSum, LpMinimize

class Material:
    def __init__(self, name, size, demand):
        self.name = name
        self.size = size
        self.demand = demand

class Truck:
    def __init__(self, name, capacity):
        self.name = name
        self.capacity = capacity

def solve_transportation(materials, trucks):
    problem = LpProblem("TransportationProblem", LpMinimize)
    transport_vars = LpVariable.dicts("Transport", ((m, t) for m in materials for t in trucks), lowBound=0, cat=LpInteger)

    # Objective function: minimize the sum of transported materials
    problem += lpSum(transport_vars[(m, t)] for m in materials for t in trucks)

    # Supply and demand constraints
    for m in materials:
        problem += lpSum(transport_vars[(m, t)] for t in trucks) == m.demand

    # Truck capacity constraints
    for t in trucks:
        problem += lpSum(transport_vars[(m, t)] * m.size for m in materials) <= t.capacity

    # Solve the problem
    problem.solve()
    print(LpStatus[problem.status])
    # Print results
    for m in materials:
        for t in trucks:
            if transport_vars[(m, t)].varValue >=0:
                print(f"{int(transport_vars[(m, t)].varValue)} units of {m.name} on {t.name}")

# Creating materials
material1 = Material("Paint", 10, 15)
material2 = Material("POP Cement", 40, 15)
material3 = Material("Cement Bags", 50, 30)


# Creating trucks
truck1 = Truck("Truck 1", 1100)
truck2 = Truck("Truck 2", 1200)

# List of materials and trucks
materials_to_transport = [material1, material2, material3]
trucks_available = [truck1, truck2]

# Solve the transportation problem
solve_transportation(materials_to_transport, trucks_available)


Optimal
15 units of Paint on Truck 1
0 units of Paint on Truck 2
15 units of POP Cement on Truck 1
0 units of POP Cement on Truck 2
6 units of Cement Bags on Truck 1
24 units of Cement Bags on Truck 2


In [ ]:
from pulp import LpProblem, LpVariable, lpSum, LpMinimize, LpInteger

class Material:
    def __init__(self, name, size):
        self.name = name
        self.size = size

class Truck:
    def __init__(self, name, capacity, cost_per_distance):
        self.name = name
        self.capacity = capacity
        self.cost_per_distance = cost_per_distance

def solve_material_transportation(materials, trucks, distances, demand):
    problem = LpProblem("ConstructionMaterialTransportation", LpMinimize)
    transport_vars = LpVariable.dicts("Transport", ((m, t) for m in materials for t in trucks), lowBound=0, cat=LpInteger)

    # Objective function: minimize total transportation cost
    problem += lpSum(transport_vars[(m, t)] * distances[m][t] * t.cost_per_distance for m in materials for t in trucks)

    # Supply and demand constraints
    for m in materials:
        problem += lpSum(transport_vars[(m, t)] for t in trucks) == demand[m]

    # Truck capacity constraints
    for t in trucks:
        problem += lpSum(transport_vars[(m, t)] * m.size for m in materials) <= t.capacity

    # Solve the problem
    problem.solve()

    if problem.status == 1:  # Check if the problem is solved optimally
        print("Optimal Solution Found:")
        for m in materials:
            for t in trucks:
                if transport_vars[(m, t)].varValue > 0:
                    print(f"{m.name} on {t.name}: {int(transport_vars[(m, t)].varValue)} units")
    else:
        print("Problem status:", problem.status)

# Creating materials
material1 = Material("Bricks", 5)
material2 = Material("Cement Bags", 2)
material3 = Material("Steel Bars", 10)

# Creating trucks
truck1 = Truck("Truck A", 20, 0.5)
truck2 = Truck("Truck B", 30, 0.6)

# Distances between materials and construction sites (in km)
distances = {
    material1: {truck1: 10, truck2: 15},
    material2: {truck1: 5, truck2: 12},
    material3: {truck1: 8, truck2: 20}
}

# Demand for materials at construction sites
demand = {
    material1: 50,
    material2: 30,
    material3: 20
}

# List of materials, trucks, and distances
materials_to_transport = [material1, material2, material3]
trucks_available = [truck1, truck2]

# Solve the construction material transportation problem
solve_material_transportation(materials_to_transport, trucks_available, distances, demand)
